In [5]:
import logging
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import Model
from cassandra.cluster import Cluster, BatchStatement
from cassandra.query import SimpleStatement

In [6]:
class PythonCassandra:
    def __init__(self):
        self.cluster = None
        self.session = None
        self.keyspace = None
        self.log = None
    def __del__(self):
        self.cluster.shutdown()
    def createsession(self):
        self.cluster = Cluster(['localhost'])
        self.session = self.cluster.connect(self.keyspace)
    def getsession(self):
        return self.session
    # How about Adding some log info to see what went wrong
    def setlogger(self):
        log = logging.getLogger()
        log.setLevel('INFO')
        handler = logging.StreamHandler()
        handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s"))
        log.addHandler(handler)
        self.log = log
    # Create Keyspace based on Given Name
    def createkeyspace(self, keyspace):
        """
        :param keyspace:  The Name of Keyspace to be created
        :return:
        """
        # Before we create new lets check if exiting keyspace; we will drop that and create new
        rows = self.session.execute("SELECT keyspace_name FROM system_schema.keyspaces")
        exists=False
        if keyspace in [row[0] for row in rows]:
            keyspace_exists = True
        if keyspace_exists is False:
            #self.log.info("dropping existing keyspace...")
            #self.session.execute("DROP KEYSPACE " + keyspace)
            self.log.info("creating keyspace...")
            self.session.execute("""
                CREATE KEYSPACE %s
                WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
                """ % keyspace)
            self.log.info("setting keyspace...")
        self.session.set_keyspace(keyspace)
    def create_table_block(self):
        c_sql = """
                CREATE TABLE IF NOT EXISTS block (block_number bigint, block_hash varchar,
                                              miner_address varchar, parent_hash varchar, receipt_tx_root varchar,
                                              state_root varchar, tx_trie_root varchar, extra_data varchar, 
                                              nonce varchar, bloom varchar, solution varchar, difficulty varchar, 
                                              total_difficulty varchar, nrg_consumed bigint, nrg_limit bigint,
                                              block_size bigint, block_timestamp bigint, block_hour tinyint, num_transactions bigint,
                                              block_time bigint, nrg_reward varchar, transaction_id bigint,
                                              transaction_list varchar,
                                              PRIMARY KEY ((block_number), block month)
                                              );
                 """
        self.session.execute(c_sql)
        self.log.info("Block Table Created !!!")
    def insert_data_block(self,myTuple):
        insert_sql = self.session.prepare("INSERT INTO  block(block_number, miner_address) VALUES (?,?)")
        batch = BatchStatement()
        batch.add(insert_sql, (1, 'LyubovK'))
        batch.add(insert_sql, myTuple)
        self.session.execute(batch)
        self.log.info('Batch Insert Completed')
        
    def select_data(self,table):
        rows = self.session.execute('select * from '+table)
        for row in rows:
            print(row)

    def update_data(self):
        pass

    def delete_data(self):
        pass
    
pc = PythonCassandra()
pc.createsession()
pc.setlogger()
pc.createkeyspace('aion')
#pc.create_table_block()
#pc.insert_data_block((1,'hello'))
#rows = pc.session.execute('SELECT MAX(block_number) FROM block where block_number = 1')
#for row in rows:
    #print(row)

2018-12-30 11:12:16,222 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:12:16,222 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:12:16,222 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes


In [3]:
'''
# DROP BLOCK
c_sql = """
                DROP TABLE IF EXISTS aion.block
                 """
pc.session.execute(c_sql)
'''

In [4]:
'''
# DROP Transaction
c_sql = """
                DROP TABLE IF EXISTS aion.transaction
                 """
pc.session.execute(c_sql)
'''

In [4]:
pc = PythonCassandra()
pc.createsession()
pc.setlogger()
'''
to_check = tuple(range(15000,15100))
rows = pc.session.execute("""SELECT block_timestamp, block_date FROM aion.block 
    where block_number in """+str(to_check))
'''
rows = pc.session.execute("""SELECT * FROM system_schema.columns 
WHERE keyspace_name = 'aion' AND table_name = 'block';""")

for row in rows:
    print(row)

2018-12-22 12:59:52,363 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes


Row(keyspace_name='aion', table_name='block', column_name='approx_nrg_reward', clustering_order='none', column_name_bytes=b'approx_nrg_reward', kind='regular', position=-1, type='float')
Row(keyspace_name='aion', table_name='block', column_name='block_date', clustering_order='none', column_name_bytes=b'block_date', kind='regular', position=-1, type='timestamp')
Row(keyspace_name='aion', table_name='block', column_name='block_day', clustering_order='none', column_name_bytes=b'block_day', kind='regular', position=-1, type='tinyint')
Row(keyspace_name='aion', table_name='block', column_name='block_month', clustering_order='none', column_name_bytes=b'block_month', kind='regular', position=-1, type='tinyint')
Row(keyspace_name='aion', table_name='block', column_name='block_number', clustering_order='none', column_name_bytes=b'block_number', kind='partition_key', position=0, type='bigint')
Row(keyspace_name='aion', table_name='block', column_name='block_size', clustering_order='none', column

In [24]:
from datetime import datetime

pc = PythonCassandra()
pc.createsession()
pc.setlogger()
to_check = tuple(range(0,2100000))

'''
rows = pc.session.execute("""SELECT block_timestamp, block_date FROM aion.block 
    where block_number in """+str(to_check))
'''
rows = pc.session.execute("""COPY aion.block TO '/tmp/data.csv';""")

for row in rows:
    print(row)

2018-12-30 11:45:12,229 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:45:12,229 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:45:12,229 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:45:12,229 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:45:12,229 [INFO] cassa

SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:0 no viable alternative at input 'COPY' ([COPY]...)">

In [19]:
from datetime import datetime

pc = PythonCassandra()
pc.createsession()
pc.setlogger()
'''
to_check = tuple(range(15000,15100))
rows = pc.session.execute("""SELECT block_timestamp, block_date FROM aionv4.block 
    where block_number in """+str(to_check))
'''
to_check = tuple(range(0,2100000))
rows = pc.session.execute("""SELECT count(*) FROM aion.transaction""")

for row in rows:
    print(row)

2018-12-30 11:32:40,293 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:32:40,293 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:32:40,293 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:32:40,293 [INFO] cassandra.policies: Using datacenter 'datacenter1' for DCAwareRoundRobinPolicy (via host '127.0.0.1'); if incorrect, please specify a local_dc to the constructor, or limit contact points to local cluster nodes
2018-12-30 11:32:40,293 [INFO] cassa

OperationTimedOut: errors={'127.0.0.1': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=127.0.0.1

2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregation query used without partition key
2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregation query used without partition key
2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregation query used without partition key
2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregation query used without partition key
2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregation query used without partition key
2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregation query used without partition key
2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregation query used without partition key
2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregation query used without partition key
2018-12-30 11:32:58,918 [WARNING] cassandra.protocol: Server warning: Aggregatio

In [26]:
import pandas as pd
def str_to_cass_ts(ts):
    #ts = pd.Timestamp(ts, unit='ns')
    ts = datetime.strptime(ts,'%Y-%m-%d')
    ts = int(ts.timestamp()*1000)
    print(ts)
    return ts


# select data using datetime
from_date = "2018-04-23"
to_date = "2018-04-25"
#convert to nanoseconds
from_date = str_to_cass_ts(from_date)
to_date = str_to_cass_ts(to_date)

qry = """SELECT count(*) from aionv4.block 
    where block_timestamp >= {} 
    and block_timestamp <= {} 
    ALLOW FILTERING
    """.format(from_date, to_date)
rows = pc.session.execute(qry)
for row in rows:
    print(row)


1524456000000
1524628800000


InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table block"